# Practice Exercises for Single Table SQL

Given a dataset and a set of questions, form the queries to answer the questions. All questions in this particular exercise only require a query to a single table. 

⚠️ *Note:* Please display minimal number of attributes (columns) needed to make sense of your answer. Not too many, not too few.


## Setup
The SQLite database for these exercises has been setup for you. For the curious, you can look at this file for the DDL commands to setup the tables and insert data.  `db_illum_ddl_5_3.sql`

Here is the schema for the database. The arrows show foreign keys, which is just fyi for now. 

![worker_project schema](worker_project_schema.png) 

In [1]:
# Make sure to always run this cell when you open this notebook
# Start the Jupyter SQL engine, connecting to a SQLite database 
%reload_ext sql 
%sql sqlite:///worker_project.db

0. In the command line, inspect the sqlite database (worker_project.db) to see what the tables and columns actually are. Any differences you observe? (Also, copy past your output in the cell below)

In [ ]:
sqlite> select * from Dept;
Accounting|101
Research|110
sqlite> select * from worker;
101|Smith|Tom|Accounting|01-Feb-1970|06-Jun-1993 |50000
103|Jones|Mary|Accounting|15-Jun-1975|20-Sep-2005|48000
105|Burns|Jane|Accounting|21-Sep-1980|12-Jun-2015|39000
110|Burns|Michael|Research|05-Apr-1977|10-Sep-2020|70000
115|Chin|Amanda|Research|22-Sep-1980|19-Jun-2022|60000
sqlite> select * from Project;
1001|Jupiter|101|300000|01-Feb-2021|50
1005|Saturn|101|400000|01-Jun-2022|35
1019|Mercury|110|350000|15-Feb-2022|40
1025|Neptune|110|600000|01-Feb-2021|45
1030|Pluto|110|380000|15-Sep-2023|50
sqlite> 
sqlite> select * from Assign;
1001|101|30|
1001|103|20|5
1005|103|20|
1001|105|30|
1001|115|20|4
1019|110|20|5
1019|115|10|4
1025|110|10|
1030|110|10|

1. Get the names of all workers in the Accounting department.

In [3]:
%%sql

SELECT firstName  || ' ' || lastName AS FullName FROM Worker WHERE deptName = "Accounting";


UsageError: Cell magic `%%sql` not found.


2. Get the name of the employee in the Research department who has the lowest salary.

In [41]:
%%sql
SELECT firstName || " " || lastName AS FullName, MIN(salary) AS Min_Salary FROM Worker WHERE deptName = "Research";

 * sqlite:///worker_project.db
Done.


FullName,Min_Salary
Amanda Chin,60000


3. Get the details of the project with the highest budget.

In [4]:
%%sql

SELECT *, MAX(budget) AS highest_budget FROM project 

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks,highest_budget
1025,Neptune,110,600000,01-Feb-2021,45,600000


## The following questions may require more than one query to answer. 
Show your work! Add as many SQL cells as needed for each question. Use comments to indicate how the output of each query is used to arrive at your answer.  

4. List out alphabetically the names of all workers working on Project 1001.

This question will requre LEFT JOIN because you have to get the firstName, lastName from the worker table, then combine the Assign table to the worker table to get the projNo.  

In [12]:

%%sql

SELECT firstName, lastName FROM worker
LEFT JOIN assign
ON worker.empid = assign.empid
WHERE projNo = 1001
ORDER BY firstName, lastName 
;

 * sqlite:///worker_project.db
Done.


firstName,lastName
Amanda,Chin
Jane,Burns
Mary,Jones
Tom,Smith


In [5]:
%%sql
SELECT * FROM worker;


 * sqlite:///worker_project.db
Done.


empId,lastName,firstName,deptName,birthDate,hireDate,salary
101,Smith,Tom,Accounting,01-Feb-1970,06-Jun-1993,50000
103,Jones,Mary,Accounting,15-Jun-1975,20-Sep-2005,48000
105,Burns,Jane,Accounting,21-Sep-1980,12-Jun-2015,39000
110,Burns,Michael,Research,05-Apr-1977,10-Sep-2020,70000
115,Chin,Amanda,Research,22-Sep-1980,19-Jun-2022,60000


In [7]:
%%sql
Select * FROM project;

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1001,Jupiter,101,300000,01-Feb-2021,50
1005,Saturn,101,400000,01-Jun-2022,35
1019,Mercury,110,350000,15-Feb-2022,40
1025,Neptune,110,600000,01-Feb-2021,45
1030,Pluto,110,380000,15-Sep-2023,50


5. Get an alphabetical list of names and corresponding ratings for all workers on any project managed by Michael Burns.

I think this will require an INNER JOIN because we are trying to combine information from three tables (Assign, Project, Worker).


In [6]:
%%sql
-- Using Joins
SELECT DISTINCT Worker.firstName, Assign.rating
FROM Assign
INNER JOIN Project ON Assign.projNo = Project.projNo
INNER JOIN Worker ON Project.projMgrId = Worker.empId
WHERE Worker.firstName = 'Michael'
ORDER BY Worker.firstName;


 * sqlite:///worker_project.db
Done.


firstName,rating
Michael,5
Michael,4
Michael,None


In [5]:
%%sql


Select DISTINCT firstName, rating
from
(Select empId, rating
from Assign 
where projNo in (SELECT projNo from Project where projMgrId = 
(Select empId from Worker where firstName == 'Michael'))) as t
INNER JOIN Worker  ON t.empId=Worker.empId



 * sqlite:///worker_project.db
Done.


firstName,rating
Michael,5
Amanda,4
Michael,None


6. For all projects starting after Feb 1, 2022, find the project number and names of all workers assigned to them. 

I think you would use a INNER JOIN because you are combining the information of 3 tables (Assign, Project, Worker).

In [10]:
%%sql
-- using joins
SELECT Project.projNo, Worker.Firstname
FROM Assign
JOIN Project ON Assign.projNo = Project.projNo
JOIN Worker ON Assign.empId = Worker.empId
WHERE Project.startDate > '2022-02-01'


 * sqlite:///worker_project.db
Done.


projNo,firstName


In [9]:
%%sql


Select projNo,Firstname from
(SELECT distinct empId,projNo
from Assign where projNo in
(select projNo from Project where startDate > '01-Feb-2022')) as T
INNER JOIN Worker on T.empId=Worker.empId


 * sqlite:///worker_project.db
Done.


projNo,firstName
1005,Mary
1019,Michael
1019,Amanda
1030,Michael


7. Find the names of employees who are not assigned as managers any project.

I think we are going to use a LEFT JOIN because we are only getting information from two tables (Worker, Project) And the Project information will be appended to the Worker table. 

In [20]:
%%sql
SELECT lastName, firstName
FROM Worker AS w
LEFT JOIN Project as p
ON w.empId = p.projMgrId
WHERE p.projMgrId IS NULL


 * sqlite:///worker_project.db
Done.


lastName,firstName
Jones,Mary
Burns,Jane
Chin,Amanda


In [11]:
%%sql

SELECT firstName, lastName FROM Worker
WHERE empId
NOT IN (SELECT DISTINCT projMgrId FROM Project)






 * sqlite:///worker_project.db
Done.


firstName,lastName
Mary,Jones
Jane,Burns
Amanda,Chin


8. Find details of any project with the string "urn" in the project name. 

I don't this this query requies a join because it only requires information from one table. 

In [21]:
%%sql

SELECT * FROM Project WHERE projName LIKE "%urn%";

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1005,Saturn,101,400000,01-Jun-2022,35


9. List out the projects from most to least efficient. (hint: the project with the lowest budget is not necessarily the one that uses time and human resources effectively.)

I don't this this query requies a join because it only requires information from one table. 

In [16]:
%%sql

SELECT * FROM Project Order by budget/expectedDurationWeeks;

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1001,Jupiter,101,300000,01-Feb-2021,50
1030,Pluto,110,380000,15-Sep-2023,50
1019,Mercury,110,350000,15-Feb-2022,40
1005,Saturn,101,400000,01-Jun-2022,35
1025,Neptune,110,600000,01-Feb-2021,45


10. Explain your reasoning for your solution to #9 (use a Markdown cell), including who should get the award for Most Efficient Project Manager.

The budget divided by the expectedDurationWeeks will result in how efficient each employee is. Since it's ordered by budget/expectedDurationWeeks, the most efficient project manager is projNo 1001. 

11. Bonus!!  A Query that you can't easily answer with just a set of queries: Get a list of project numbers with name and start dates, of all the projects that have the same start date as another. 
*you don't have to answer this, but think about it!*


I think this query would require joining the Project table to itself (self join). This would require using LEFT JOIN on itself to get the list of project number/names/start dates with the same start date as another project. 

## Submission:

Commit and push your changes to this document by the deadline, and share a link to the repository in the Moodle assignment.  

People who helped:

Isaac,
Chesky,
Molly,
Anish,
Daize